In [7]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [9]:
def relu(x):
    return np.maximum(0,x)

In [10]:
def identify_function(x):
    return x

In [11]:
def softmax(a): # 출력층의 활성화 함수. 주로 학습시킬때만 사용.
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

### 행렬의 곱 예제

In [12]:
x = np.array([1, 2])
w = np.array([[1, 3, 5], [2, 4, 6]])
y = np.dot(x, w)
print(y)

[ 5 11 17]


### 신경망 구현

#### 입력층에서 각 층으로 신호 전달

In [13]:
x = np.array([1.0, 0.5]) # input values

W1 = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]]) # W. weight
b1 = np.array([0.1, 0.2, 0.3]) # b. bias

a1 = np.dot(x, W1) + b1 # a. result
z1 = sigmoid(a1) # z. rectified by sigmoid

#### 1층에서 2층으로 신호 전달

In [14]:
W2 = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
b2 = np.array([0.1, 0.2])

a2 = np.dot(z1, W2) + b2
z2 = sigmoid(a2)

#### 2층에서 출력층으로 신호 전달

In [15]:
W3 = np.array([[0.1, 0.3], [0.2, 0.4]])
b3 = np.array([0.1, 0.2])

a3 = np.dot(z2, W3) + b3
y = identify_function(a3) # activation func

#### 구현 정리

In [16]:
def init_network():
    network = {}
    network['W1'] = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['W2'] = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
    network['b2'] = np.array([0.1, 0.2])
    network['W3'] = np.array([[0.1, 0.3], [0.2, 0.4]])
    network['b3'] = np.array([0.1, 0.2])
    
    return network

def forword(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = identify_function(a3)
    
    return y

network = init_network()
x = np.array([0.1, 0.5])
y = forword(network, x)
print(y) # [0.31234736 0.6863161 ]

[0.31234736 0.6863161 ]


### MNIST 예제

In [17]:
import sys, os
sys.path.append(os.path.abspath('..'))
import pickle
from dataset.mnist import load_mnist
from common.functions import sigmoid, softmax

(x_train, y_train), (x_test, y_test) = load_mnist(flatten=True, normalize=False)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [18]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

In [19]:
def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

In [20]:
def predict(network, x):
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, w3) + b3
    y = softmax(a3)

    return y

In [21]:
x, t = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352
